# Using Decorators for Traceback Handling

Here is an example of how to use decorators to handle tracebacks in Python and change every functions behavior without modifying their code directly.

## Example

Change `main` function to use a decorator for traceback handling as `cm_main` with using context manager and using `ErrorCatcher` class with use of wrapper function (`exec_in_safe_mode`).


## Simple explanation of method

1. Define a decorator function that wraps the target function.
2. Inside the wrapper, use a try-except block to catch exceptions.
3. In the except block, handle the traceback as needed (e.g., logging, printing, etc.).
4. Apply the decorator to any function where you want to handle tracebacks consistently.

## Feature works

- Show `file_path` for each function where error occurs.


In [81]:
import traceback
# from time import time_ns as time

In [82]:
def div(a, b):
    """clean division"""
    c = a / b
    return c

In [83]:
def simple_func(inp):
    """a simple function that raises an exception"""
    print(f"==>> inp: {inp}")
    # 4
    # 5
    # 6
    # 7
    # 8
    # 9
    # 10   
    x = div(2, 3)
    y= div(2, 0)  # This will raise a ZeroDivisionError
    return x + y

In [84]:
def main():
    try:
        result = simple_func(inp='started')
        print("Result:", result)
    except Exception as e:
        print("An error occurred:", e)

In [85]:
main()

==>> inp: started
An error occurred: division by zero


In [86]:
def new_main():
    try:
        result = simple_func(inp='started')
        print("Result:", result)
    except Exception as e:
        print("An error occurred:", e)
        traceback.print_exc()

In [87]:
new_main()

==>> inp: started
An error occurred: division by zero


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_10060\2808983410.py", line 3, in new_main
    result = simple_func(inp='started')
  File "C:\Users\user\AppData\Local\Temp\ipykernel_10060\39230.py", line 12, in simple_func
    y= div(2, 0)  # This will raise a ZeroDivisionError
  File "C:\Users\user\AppData\Local\Temp\ipykernel_10060\1626019231.py", line 3, in div
    c = a / b
        ~~^~~
ZeroDivisionError: division by zero


In [88]:
def better_main():
    try:
        result = simple_func(inp='started')
        print("Result:", result)
    except Exception as e:
        print(e)
        tb = traceback.extract_tb(e.__traceback__)
        for _, lineno, name, line in tb:
                    print(f"{name.rjust(11, ' ')}({('#'+str(lineno)).rjust(5, ' ')}): {line}")

In [89]:
better_main()  # readable main error traceback in console

==>> inp: started
division by zero
better_main(   #3): result = simple_func(inp='started')
simple_func(  #12): y= div(2, 0)  # This will raise a ZeroDivisionError
        div(   #3): c = a / b


In [90]:
class TracebackCatcher:
    def __enter__(self):
        # Nothing special needed at entry
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        # If no exception happened, do nothing
        if exc_type is None:
            return False

        print(exc_value)

        tb = traceback.extract_tb(exc_tb)
        for _, lineno, name, line in tb:

            print(
                f"{name}(#{lineno})".rjust(17, ' ') +
                f": {line}"
            )

        # IMPORTANT:
        # Returning True means: "I handled the exception, don't raise it"
        return True

In [91]:
def useful_main():
    with TracebackCatcher():
        result = simple_func(inp='started')
        print("Result:", result)

In [92]:
useful_main()  # using with block

==>> inp: started
division by zero
  useful_main(#3): result = simple_func(inp='started')
 simple_func(#12): y= div(2, 0)  # This will raise a ZeroDivisionError
          div(#3): c = a / b


In [93]:
class ErrorCatcher:

    METHOD = 1
    FUN_MAX_CHAR = 15
    def __enter__(self):
        # nothing special needed on entry
        #_#/\ self.start = time()
        return None

    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is not None:
            if self.METHOD == 0:
                print("\n⚠️ Exception caught (but not raised):\n")
                # formatted traceback
                tb_lines = traceback.format_exception(exc_type, exc_value, exc_tb)
                for line in tb_lines:
                    print(line, end="")
            elif self.METHOD == 1:
                print(f"\n⚠️ Exception caught (but not raised): {exc_value}\n")
                # formatted traceback
                tb = traceback.extract_tb(exc_tb)
                for _, lineno, name, line in tb:
                    print(
                        f"{name}(#{lineno})".rjust(self.FUN_MAX_CHAR, ' ') + f': "{line}"'
                    )

            # return True → suppress exception
            #_#/\ print(f"Catched in {(time() - self.start)/1e3:.1f}(ms).")
            return True

        # no exception → do nothing special
        return False


In [94]:
def exec_in_safe_mode(func):
    
    def wrapper(*args, **kwargs):
        with ErrorCatcher():
            return func(*args, **kwargs)
    return wrapper


In [95]:
@exec_in_safe_mode
# def cm_main(i):
def cm_main():
    # print(f"==>> i: {i}")
    result = simple_func(inp='started')
    print("Result:", result)


In [96]:
# tm = time()
cm_main()
# print(f"==>> duration: {(time() - tm)/1e3:.3f}(ms)")

==>> inp: started

⚠️ Exception caught (but not raised): division by zero

    wrapper(#5): "return func(*args, **kwargs)"
    cm_main(#5): "result = simple_func(inp='started')"
simple_func(#12): "y= div(2, 0)  # This will raise a ZeroDivisionError"
        div(#3): "c = a / b"
